# Topic Modelling: A Comparison of Evaluation Methods

- Wilfrid Laurier University (Winter 2018)
- CS640 - Introduction to Machine Learning
- Ryan Kazmerik (175826410)

## Overview
This notebook includes an experiment to compare intrinsic vs. extrinsic evaluation methods when considering topic modelling and contains the following sections:

1. [Dataset](#dataset)
2. [Vectorizing Features](#features)
3. [Fitting Models](#models)
4. [Viewing Topic Terms](#topterms)
5. [Extrinsic Evaluation](#extrinsic)
6. [Intrinsic Evaluation](#intrinsic)
7. [Comparing Evaluations](#evaluations)
8. [Visualizing Results](#visualizations)

## <a id='dataset'>Dataset
### 1. 20newsgroups
The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics and has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering.

Let's import the 20newsgroups dataset:

In [5]:
from __future__ import print_function
import warnings; warnings.simplefilter('ignore')

from sklearn.datasets import fetch_20newsgroups

categories = [
    'rec.sport.baseball',
    'sci.space',
    'talk.religion.misc'
]

# ds1 = 20-newsgroups dataset
ds1 = fetch_20newsgroups(subset='all', categories=categories, shuffle=True,  
                         random_state=1, remove=('headers','footers','quotes'))

print ('20-NEWSGROUPS DATASET:')
print ('  Documents=', len(ds1.data))
print ('  Categories=', len(ds1.target_names))
print ()
print ('SAMPLE DOCUMENT: ')
print (ds1.data[5])

20-NEWSGROUPS DATASET:
  Documents= 2609
  Categories= 3

SAMPLE DOCUMENT: 

Actually, Hiten wasn't originally intended to go into lunar orbit at all,
so it indeed didn't have much fuel on hand.  The lunar-orbit mission was
an afterthought, after Hagoromo (a tiny subsatellite deployed by Hiten
during a lunar flyby) had a transmitter failure and its proper insertion
into lunar orbit couldn't be positively confirmed.

It should be noted that the technique does have disadvantages.  It takes
a long time, and you end up with a relatively inconvenient lunar orbit.
If you want something useful like a low circular polar orbit, you do have
to plan to expend a certain amount of fuel, although it is reduced from
what you'd need for the brute-force approach.


## <a id='features'>Vectorizing Features
The text in the documents must be parsed to remove stop words (tokenization) and the words need to be encoded as floating point values to be used as input for our clustering algorithm (vectorization).

Let's create some feature vectors for our datasets:

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text 

my_additional_stop_words = ['like','don']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

# Create some vectorizers for each algorithm
km_vectorizer = TfidfVectorizer(max_df=0.2, min_df=2, 
                     max_features=1000, stop_words='english')

lda_vectorizer = CountVectorizer(max_df=0.95, min_df=2, analyzer='word',
                     max_features=1000, stop_words=stop_words, token_pattern = r'\b[a-zA-Z]{3,}\b')

plsa_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, 
                     max_features=1000, stop_words='english')

# Generate feature sets for all 3 algorithms
fs1 = km_vectorizer.fit_transform(ds1.data)
fs2 = lda_vectorizer.fit_transform(ds1.data)
fs3 = plsa_vectorizer.fit_transform(ds1.data)

print ('20-newsgroups features:')
print ('   Num features:', fs1.shape[1])
print ('   Non-zero components:', fs1.nnz / float(fs1.shape[0]))

20-newsgroups features:
   Num features: 1000
   Non-zero components: 25.6561901112


## <a id='models'>Fitting Models (Kmeans, LDA, PLSA)
Three algorithms are used to cluster the results including a standard implementation of Kmeans, Non-negative Matrix Factorization is applied with the generalized Kullback-Leibler divergence which is equivalent to Probabilistic Latent Semantic Analysis (PLSA).

Let's cluster our feature sets:

In [14]:
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation, NMF
from time import time

n_components = 3

# Fit the kMeans model KM1=20-newsgroups
t0 = time()
KM = KMeans(n_clusters=n_components, init='k-means++', max_iter=100, 
            n_init=1, verbose=0)
print ('Fitting the kMeans model...')

KM1 = KM.fit(fs1)

print ('   done in:', (time()-t0))
print ()

# Fit the LDA model LDA1=20-newsgroups
t0 = time()
LDA = LatentDirichletAllocation(n_components=n_components, max_iter=5,
            learning_method='online', learning_offset=50., random_state=0)
print ('Fitting the LDA model...')     
LDA1 = LDA.fit(fs2)

print ('   done in:', (time()-t0))
print ()

# Fit the PLSA model PLSA1=20-newsgroups
t0 = time()
PLSA = NMF(n_components=n_components, random_state=1, beta_loss='kullback-leibler', 
           solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5)
print ('Fitting the PLSA model...')

PLSA1 = PLSA.fit(fs3)

print ('   done in:', (time()-t0))

Fitting the kMeans model...
   done in: 1.31401109695

Fitting the LDA model...
   done in: 7.95743703842

Fitting the PLSA model...
   done in: 0.85965013504


## <a id='topterms'>Viewing Topic Terms

In [15]:
n_terms = 10
terms1 = km_vectorizer.get_feature_names()
terms2 = lda_vectorizer.get_feature_names()
terms3 = plsa_vectorizer.get_feature_names()

print("Kmeans - Topic model:")

order_centroids = KM1.cluster_centers_.argsort()[:, ::-1]
for i in range(3):
    print(" Topic %d:" % i, end='')
    for ind in order_centroids[i, :n_terms]:
        print(' %s' % terms1[ind], end='')
    print ()
print()
   
    
print("LDA - Topic model:")

for topic_idx, topic in enumerate(LDA1.components_):
    message = " Topic %d: " % topic_idx
    message += " ".join([terms2[i]
                    for i in topic.argsort()[:-n_terms - 1:-1]])
    print(message)
print()


print("pLSA - Topic model:")

for topic_idx, topic in enumerate(PLSA1.components_):
    message = " Topic %d: " % topic_idx
    message += " ".join([terms3[i]
                    for i in topic.argsort()[:-n_terms - 1:-1]])
    print(message)
print()


Kmeans - Topic model:
 Topic 0: god know people does say jesus time did want good
 Topic 1: year game team games baseball hit good runs players braves
 Topic 2: space nasa shuttle launch orbit moon earth mission program hst

LDA - Topic model:
 Topic 0: year game team good think games baseball hit won just
 Topic 1: god people jesus just know say think does bible did
 Topic 2: space nasa earth launch data time shuttle orbit moon spacecraft

pLSA - Topic model:
 Topic 0: think time like just don year years good know way
 Topic 1: space use like nasa earth orbit thanks new shuttle used
 Topic 2: people god say think don just way did know read



## <a id='extrinsic'>Extrinsic Evaluation
Due to the lack of precise accuracy measurments, often topic modelling is evaluated using an extrinsic measure. That is, a baseline set of results are provided by a human test participant or subject matter expert. For our baseline result set, we will be using a list of 10 human-generated terms for each topic - for more information on the design of this task please see the research paper related to this experiment.

### Part 1 : Co-occurance Measure
The first part of the extrinsic evaluation will measure the rate of co-occurance between our baseline topics and the modeled topics. Each model will be assigned a normalized score between 0 - 1, a threshold of >0.5 has been assigned to qualify as a performant topic model. 

Topic | Method | t1 | t2 | t3 | t4 | t5 | t6 | t7 | t8 | t9 | t10 | **CO** | **CS (CO/nt)**
------|-------|----|----|----|----|----|----|----|----|----|----
0 | **K-means**|space|nasa|shuttle|launch|orbit|moon|earth|mission|hst|program|**8**|**0.8**
0 | **LDA**|space|nasa|earth|launch|data|shuttle|orbit|spacecraft|solar|moon|**8**|**0.8**
0 |**pLSA**|space|nasa|orbit|new|shuttle|earth|use|program|launch|used|**6**|**0.6**
0 |**Baseline**|space|mission|solar|exploration|moon|launch|nasa|orbit|earth|shuttle|**-**|**-**
| | | | | | | | | | | | |
1 | **K-means**|god|jesus|people|christian|bible|know|koresh|right|christians|say|**3**|**0.3**
1 | **LDA**|people|read|god|just|think|know|christian|say|does|jesus|**3**|**0.3**
1 |**pLSA**|people|god|say|think|don|just|way|did|know|read|**1**|**0.1**
1 |**Baseline**|religion|christian|jesus|muslim|jew|god|faith|view|heaven|scripture|**-**|**-**
| | | | | | | | | | | | |
2 | **K-means**|year|game|team|games|baseball|hit|good|players|runs|braves|**4**|**0.4**
2 | **LDA**|year|game|team|games|baseball|hit|won|runs|players|league|**5**|**0.5**
2 |**pLSA**|think|time|like|just|don|year|years|game|know|way|**1**|**0.1**
2 |**Baseline**|baseball|pitch|bat|team|score|league|mlb|games|statistics|players|**-**|**-**

In [16]:
#Calculate overall co-occurance score for each method
n_topics = 3

km_co_score = (0.8+0.3+0.4) / n_topics
lda_co_score = (0.8+0.3+0.5) / n_topics
plsa_co_score = (0.6+0.1+0.1) / n_topics

print ('Co-occurance measures (threshold > .5)')
print (' K-means=', km_co_score, 'fail')
print (' LDA=', lda_co_score, 'pass')
print (' pLSA=', plsa_co_score, 'fail')

Co-occurance measures (threshold > .5)
 K-means= 0.5 fail
 LDA= 0.533333333333 pass
 pLSA= 0.266666666667 fail


## Part 2 : Probability Measure
For the second part of our extrinsic evaluation, we will select the algorithms above our co-currance threshold from part 1 (LDA) and calculate the probabilities of each term in the topic model against our baseline terms.

Topic | Term | Probability | Co-occurance | PS
------|------|-------------|--------------|------
0|space|1262.39|1|1262.39
0|nasa|488.90|1|488.90
0|earth|418.79|1|418.79
0|launch|348.33|1|348.33
0|data|282.03|0|0
0|shuttle|286.92|1|286.92
0|orbit|284.63|1|284.63
0|spacecraft|260.17|0|0
0|solar|258.66|1|258.66
0|moon|262.45|1|262.45
1|people|709.44|0|0
1|don|693.71|0|0
1|god|664.35|1|664.35
1|just|648.34|0|0
1|think|623.70|0|0
1|know|570.54|0|0
1|christian|209.42|1|209.42
1|say|449.83|0|0
1|does|434.53|0|0
1|jesus|416.45|1|416.45
2|year|487.19|0|0
2|game|428.48|1|428.48
2|team|319.71|1|319.71
2|games|276.94|0|0
2|baseball|260.40|1|260.40
2|hit|240.49|0|0
2|won|207.12|0|0
2|runs|202.95|0|0
2|players|186.44|1|186.44
2|league|182.51|1|182.51

In [17]:
#Calculate probability measure for all topics
n_terms = 10

topic0_ps = (1262.39+488.90+418.79+348.33+286.92+284.63+258.66+262.45)
topic1_ps = (664.35+209.42+416.45)
topic2_ps = (428.48+319.71+260.40+186.44+182.51)

print ('Probability scores:')
print (' Topic 0=', topic0_ps)
print (' Topic 1=', topic1_ps)
print (' Topic 2=', topic2_ps)
print ('Probability measure=', (topic0_ps/n_terms)+(topic1_ps/n_terms)+(topic2_ps/n_terms))


Probability scores:
 Topic 0= 3611.07
 Topic 1= 1290.22
 Topic 2= 1377.54
Probability measure= 627.883


## <a id='intrinsic'>Intrinsic Evaluation
The most widely used instrinsic evaluation method is known as **Perplexity** which can be thought of as the inverse per-word likelihood of our LDA model. The accurate way to calculate perplexity is using a hold-out test set to compare against our trained model. 

Let's evaluate our LDA model using perplexity:

In [18]:
# Create test and training data sets
ds_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True,  
                         random_state=1, remove=('headers','footers','quotes'))

ds_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True,  
                         random_state=1, remove=('headers','footers','quotes'))

# Vectorize the datasets
fs_train = lda_vectorizer.fit_transform(ds_train.data)
fs_test = lda_vectorizer.fit_transform(ds_test.data)

# Fit the LDA model to the training features
t0 = time()
print ('Fitting the LDA model...')     

LDA_train = LDA.fit(fs_train)
print ('   done in:', (time()-t0))
print ()

# Calculate model perplexity using the test set
perplexity = LDA_train.perplexity(fs_test)
print ('Perplexity= ',perplexity)

Fitting the LDA model...
   done in: 5.20609903336

Perplexity=  1435.8732953881436


## <a id='evaluations'>Comparing Evaluation Methods
Now that we have generated both extrinsic and intrinsic evaluation methods, let's compare the measurements as we increase the number of features to see how they correspond to each other:

In [25]:
f_array = [50,100,250,500,750,1000,1250,1500]
x_data = []
y1_data = []
y2_data = []

for n_features in f_array:
    
    # Create the vectorizer and generate features
    e_vectorizer = CountVectorizer(max_df=0.95, min_df=2, analyzer='word',
                         max_features=n_features, stop_words='english')

    e_fs = e_vectorizer.fit_transform(ds1.data)

    # Fit the LDA model for evaluation
    t0 = time()

    print ('Fitting the LDA model w/',n_features,'features...')     
    e_LDA = LDA.fit(e_fs)

    print ('   done in:', (time()-t0))

    # Get the vectorizer terms and the base class terms
    v_terms = e_vectorizer.get_feature_names()
    bl_terms_0 = ['space','mission','solar','exploration','moon','launch','nasa','orbit','earth','shuttle']
    bl_terms_1 = ['religion','christian','jesus','muslim','jew','god','faith','view','heaven','scripture']
    bl_terms_2 = ['year','game','team','games','baseball','hit','won','runs','players','league']

    # Calculate the total probability
    total_prob = 0;
    n_bl_terms = len(bl_terms_0) + len(bl_terms_1) + len(bl_terms_2)

    for i, term in enumerate (v_terms):
        if (term in bl_terms_0): total_prob += e_LDA.components_[0][i]
        if (term in bl_terms_1): total_prob += e_LDA.components_[1][i]
        if (term in bl_terms_2): total_prob += e_LDA.components_[2][i]

    prob_measure = total_prob / n_bl_terms
    print ('Probability measure=', prob_measure)

    # Calculate the perplexity score
    e_train = e_vectorizer.fit_transform(ds_train.data)
    e_test = e_vectorizer.fit_transform(ds_test.data)

    e_LDA_train = LDA.fit(e_train)

    perplexity = e_LDA_train.perplexity(e_test)%len(f_array) 
    print ('Perplexity= ', perplexity)
    print ()
    
    x_data.append(n_features)
    y1_data.append(perplexity)
    y2_data.append(prob_measure)

Fitting the LDA model w/ 50 features...
   done in: 4.88487792015
Probability measure= 7.586414930619255
Perplexity=  3.1585454058280646

Fitting the LDA model w/ 100 features...
   done in: 5.68527603149
Probability measure= 6.771777302664992
Perplexity=  4.7780878600270995

Fitting the LDA model w/ 250 features...
   done in: 7.37557601929
Probability measure= 133.576141356697
Perplexity=  2.681242748050863

Fitting the LDA model w/ 500 features...
   done in: 6.27795100212
Probability measure= 269.4394083745913
Perplexity=  7.009971483212325

Fitting the LDA model w/ 750 features...
   done in: 7.51289200783
Probability measure= 279.70652470326814
Perplexity=  4.979713881835551

Fitting the LDA model w/ 1000 features...
   done in: 7.04675006866
Probability measure= 279.99911393702206
Perplexity=  4.429652366836535

Fitting the LDA model w/ 1250 features...
   done in: 7.66306614876
Probability measure= 140.01920023577514
Perplexity=  1.6217119420118706

Fitting the LDA model w/ 150

## <a id='visualizations'>Visualizing the Results
We can see that as the number of features increases, the probability measure seems to increase, and the perplexity decreases, let's plot these on a line chart to further understand the relationship and identify the elbow point.

In [6]:
import plotly.plotly as py
import plotly.graph_objs as go

# Create traces
perp = go.Scatter(x=x_data, y=y1_data, mode='lines+markers', name='Perplexity')
prob = go.Scatter(x=x_data, y=y2_data, mode='lines+markers', name='Probability', yaxis='y2')
data = [perp, prob]

layout = go.Layout(title = 'Perplexity/Probability vs. No. Features',
    yaxis = dict(title='Perplexity'),
    yaxis2 = dict(title='Probability', overlaying='y', side='right'),
    xaxis = dict(title='No. Features'),
    autosize=False, width=1200, height=500
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

NameError: name 'x_data' is not defined

As we can see in the results above, the perplexity measure would indicate our model is ideal at 100 features, while the probability measure indicates it is ideal at 1000 features.

Let's see what our topic model looks like at 100 and 1000 features:

In [27]:
import pyLDAvis
import pyLDAvis.sklearn
warnings.simplefilter('ignore')

g_vectorizer = CountVectorizer(max_df=0.95, min_df=2, analyzer='word',
    max_features=1000, stop_words=stop_words, token_pattern = r'\b[a-zA-Z]{3,}\b')

fs = g_vectorizer.fit_transform(ds1.data)
lda = LDA.fit(fs)
    
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, fs, g_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      39.169691        1       1 -0.119541  0.116747
1      34.974303        1       2 -0.068205 -0.140163
0      25.856006        1       3  0.187747  0.023416, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
825   Default  1213.000000       space  1213.000000  30.0000  30.0000
344   Default   667.000000         god   667.000000  29.0000  29.0000
327   Default   467.000000        game   467.000000  28.0000  28.0000
994   Default   695.000000        year   695.000000  27.0000  27.0000
433   Default   419.000000       jesus   419.000000  26.0000  26.0000
567   Default   468.000000        nasa   468.000000  25.0000  25.0000
882   Default   395.000000        team   395.000000  24.0000  24.0000
328   Default   295.000000       games   295.000000  23.0000  23.0000
79    Default   281.000000    baseball   281.000000  22.0000  22.0000
469   Default   336.000000      launch   336.000000  21.0000  21.0000
383   Default   280.000000         hit   280.000000  20.0000  20.0000
94    Default   279.000000       bible   279.000000  19.0000  19.0000
650   Default   229.000000     players   229.000000  18.0000  18.0000
756   Default   222.000000        runs   222.000000  17.0000  17.0000
979   Default   270.000000         won   270.000000  16.0000  16.0000
797   Default   275.000000     shuttle   275.000000  15.0000  15.0000
475   Default   198.000000      league   198.000000  14.0000  14.0000
976   Default   192.000000         win   192.000000  13.0000  13.0000
601   Default   273.000000       orbit   273.000000  12.0000  12.0000
620   Default   745.000000      people   745.000000  11.0000  11.0000
432   Default   223.000000     jehovah   223.000000  10.0000  10.0000
780   Default   174.000000      season   174.000000   9.0000   9.0000
208   Default   290.000000        data   290.000000   8.0000   8.0000
561   Default   255.000000        moon   255.000000   7.0000   7.0000
639   Default   164.000000    pitching   164.000000   6.0000   6.0000
148   Default   208.000000      christ   208.000000   5.0000   5.0000
149   Default   211.000000   christian   211.000000   4.0000   4.0000
649   Default   159.000000      player   159.000000   3.0000   3.0000
826   Default   249.000000  spacecraft   249.000000   2.0000   2.0000
250   Default   484.000000       earth   484.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
475    Topic3   192.583562      league   198.409803   1.3228  -4.9266
976    Topic3   186.913725         win   192.671782   1.3223  -4.9565
650    Topic3   221.046215     players   229.155917   1.3166  -4.7888
852    Topic3   104.184888       stats   107.006471   1.3259  -5.5410
385    Topic3   100.211355      hitter   102.871753   1.3264  -5.5799
565    Topic3   111.634693      morris   115.064225   1.3224  -5.4719
756    Topic3   211.428073        runs   222.788857   1.3003  -4.8333
267    Topic3    89.439371         era    92.189955   1.3223  -5.6936
383    Topic3   252.507307         hit   280.890304   1.2461  -4.6557
647    Topic3   156.067761        play   169.845633   1.2680  -5.1369
882    Topic3   336.756572        team   395.445424   1.1920  -4.3678
979    Topic3   237.436263         won   270.360171   1.2228  -4.7173
387    Topic3    96.642678     hitting   101.132472   1.3072  -5.6162
994    Topic3   513.569440        year   695.776562   1.0490  -3.9458
753    Topic3   181.169045         run   211.858380   1.1961  -4.9877
883    Topic3   106.475727       teams   114.600317   1.2791  -5.5193
71     Topic3   150.184993     average   173.785695   1.2067  -5.1753
74     Topic3   129.244190        ball   145.825119   1.2319  -5.3255
501    Topic3   147.620580        lost   172.915936   1.1945  -5.1925
390    Topic3   132.889253        home   152.52